# Task 1: data preparation and customer analytics

## Objective

1. Analyze transaction and customer data to identify trends and inconsistencies
    - Retail Analytics
    - Understand the types of customers who purchase Chips and, 
    - Their purchasing behaviour within the region.
2. Develop Metrics and examine sales drivers to gain insights into overall sales performance
3. Create visualization and prepare findings to formulate a clear recommendation for the clien's strategy


In [124]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Step 1: Exploratory Data Analysis

In [125]:
# Import Customer and transaction data for wrangling

customerData = pd.read_csv("./QVI_purchase_behaviour.csv")
transactionData = pd.read_excel("./QVI_transaction_data.xlsx")

In [126]:
# Analyzing the data
customerData.head()
transactionData.head()

,DATE,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES
0,43390,1,1000,1,5,Natural Chip Compny SeaSalt175g,2,6.0
1,43599,1,1307,348,66,CCs Nacho Cheese 175g,3,6.3
2,43605,1,1343,383,61,Smiths Crinkle Cut Chips Chicken 170g,2,2.9
3,43329,2,2373,974,69,Smiths Chip Thinly S/Cream&Onion 175g,5,15.0
4,43330,2,2426,1038,108,Kettle Tortilla ChpsHny&Jlpno Chili 150g,3,13.8


In [127]:
## Inspect size of dataset
# transactionData.shape

## Understand the dataset by showing the number of records in each column, type of data, whether any values are missing and how much memory the dataset uses.
transactionData.info()

## Statistical summary of data: count, mean, standard deviation, minimum and quartiles for each numerical column
# transactionData.describe()

## Inspect list of Columns
# transactionData.columns.to_list()

## Check Null Values
# transactionData.isnull().sum()

## Checking for the duplicate values
# transactionData.nunique()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264836 entries, 0 to 264835
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   DATE            264836 non-null  int64  
 1   STORE_NBR       264836 non-null  int64  
 2   LYLTY_CARD_NBR  264836 non-null  int64  
 3   TXN_ID          264836 non-null  int64  
 4   PROD_NBR        264836 non-null  int64  
 5   PROD_NAME       264836 non-null  object 
 6   PROD_QTY        264836 non-null  int64  
 7   TOT_SALES       264836 non-null  float64
dtypes: float64(1), int64(6), object(1)
memory usage: 16.2+ MB


In [128]:
# "DATE" is an Excel serial date. Convert to standard date
transactionData['DATE'] = pd.to_datetime(transactionData['DATE'], origin='1899-12-30', unit='D')
# transactionData['DATE'].head()

Analysis of Product Names `PROD_NAME`

In [129]:
# Get all word occurrence in products name and add them to a dataframe # Split all unique product names into words
productWords = pd.DataFrame(
    {'words': "".join(transactionData['PROD_NAME'].unique()).split()}
    )

# Remove all words with mixed chars and Keep only alphabetic words (remove anything with numbers or symbols)
cleanWords = productWords[productWords['words'].str.fullmatch(r'[a-zA-Z]+')]


In [130]:
# cleanWords.value_counts()
# Create a boolean column 'SALSA' that checks if 'salsa' is in the product name
transactionData['SALSA'] = transactionData['PROD_NAME'].str.lower().str.contains("salsa")

# Filter out rows where 'SALSA' is True and drop the 'SALSA' column
salsa_filter = transactionData['SALSA'] == False

transactionData = transactionData[salsa_filter]
# transactionData = transactionData[transactionData['SALSA'] == False].drop(columns=['SALSA'], inplace=True)

transactionData.drop(columns=['SALSA'], inplace=True)
transactionData.describe()

,DATE,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_QTY,TOT_SALES
count,246742,246742.000000,2.467420e+05,2.467420e+05,246742.000000,246742.000000,246742.000000
mean,2018-12-30 01:19:01.211467520,135.051098,1.355310e+05,1.351311e+05,56.351789,1.908062,7.321322
min,2018-07-01 00:00:00,1.000000,1.000000e+03,1.000000e+00,1.000000,1.000000,1.700000
25%,2018-09-30 00:00:00,70.000000,7.001500e+04,6.756925e+04,26.000000,2.000000,5.800000
50%,2018-12-30 00:00:00,130.000000,1.303670e+05,1.351830e+05,53.000000,2.000000,7.400000
75%,2019-03-31 00:00:00,203.000000,2.030840e+05,2.026538e+05,87.000000,2.000000,8.800000
max,2019-06-30 00:00:00,272.000000,2.373711e+06,2.415841e+06,114.000000,200.000000,650.000000
std,NaN,76.787096,8.071528e+04,7.814772e+04,33.695428,0.659831,3.077828


In [137]:
qty_filter = transactionData['PROD_QTY'] == 200
# transactionData[qty_filter]['LYLTY_CARD_NBR'].iloc[0]
outl_cust_id = transactionData[qty_filter]['LYLTY_CARD_NBR'].iloc[0]

outl_filter = transactionData['LYLTY_CARD_NBR'] != outl_cust_id

transactionData = transactionData[outl_filter]

transactionData.describe()

,DATE,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_QTY,TOT_SALES
count,246740,246740.000000,2.467400e+05,2.467400e+05,246740.000000,246740.000000,246740.000000
mean,2018-12-30 01:18:58.448569344,135.050361,1.355303e+05,1.351304e+05,56.352213,1.906456,7.316113
min,2018-07-01 00:00:00,1.000000,1.000000e+03,1.000000e+00,1.000000,1.000000,1.700000
25%,2018-09-30 00:00:00,70.000000,7.001500e+04,6.756875e+04,26.000000,2.000000,5.800000
50%,2018-12-30 00:00:00,130.000000,1.303670e+05,1.351815e+05,53.000000,2.000000,7.400000
75%,2019-03-31 00:00:00,203.000000,2.030832e+05,2.026522e+05,87.000000,2.000000,8.800000
max,2019-06-30 00:00:00,272.000000,2.373711e+06,2.415841e+06,114.000000,5.000000,29.500000
std,NaN,76.786971,8.071520e+04,7.814760e+04,33.695235,0.342499,2.474897


In [139]:
transactionData.describe()

,DATE,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_QTY,TOT_SALES
count,246740,246740.000000,2.467400e+05,2.467400e+05,246740.000000,246740.000000,246740.000000
mean,2018-12-30 01:18:58.448569344,135.050361,1.355303e+05,1.351304e+05,56.352213,1.906456,7.316113
min,2018-07-01 00:00:00,1.000000,1.000000e+03,1.000000e+00,1.000000,1.000000,1.700000
25%,2018-09-30 00:00:00,70.000000,7.001500e+04,6.756875e+04,26.000000,2.000000,5.800000
50%,2018-12-30 00:00:00,130.000000,1.303670e+05,1.351815e+05,53.000000,2.000000,7.400000
75%,2019-03-31 00:00:00,203.000000,2.030832e+05,2.026522e+05,87.000000,2.000000,8.800000
max,2019-06-30 00:00:00,272.000000,2.373711e+06,2.415841e+06,114.000000,5.000000,29.500000
std,NaN,76.786971,8.071520e+04,7.814760e+04,33.695235,0.342499,2.474897
